In [1]:
import numpy as np
import networkx as nx
import pandas as pd
import re
import scipy as sp
import matplotlib.pyplot as plt

In [2]:
cits = pd.read_csv('aps-dataset-citations-2020.csv', sep=',')

In [3]:
df = cits[cits['citing_doi'].str.contains('PhysRevLett')]
df = df[df['cited_doi'].str.contains('PhysRevLett')]

In [4]:
num_mask = "\.\d{1,2}\."
num_mask='\.[0-9]\.|\.[1-4][0-9]\.'
df = df[df['cited_doi'].str.contains(num_mask)]
df = df[df['citing_doi'].str.contains(num_mask)]

In [5]:
df

,citing_doi,cited_doi
328715,10.1103/PhysRevLett.2.284,10.1103/PhysRevLett.1.9
328721,10.1103/PhysRevLett.1.127,10.1103/PhysRevLett.1.9
328729,10.1103/PhysRevLett.14.611,10.1103/PhysRevLett.1.12
328735,10.1103/PhysRevLett.24.526,10.1103/PhysRevLett.1.12
328741,10.1103/PhysRevLett.11.333,10.1103/PhysRevLett.1.16
...,...,...
1803616,10.1103/PhysRevLett.49.1681,10.1103/PhysRevLett.49.1680
1806750,10.1103/PhysRevLett.49.1776,10.1103/PhysRevLett.49.1772
1806860,10.1103/PhysRevLett.49.1799,10.1103/PhysRevLett.49.1798
1808836,10.1103/PhysRevLett.49.1865,10.1103/PhysRevLett.49.1861


In [62]:
g = nx.from_pandas_edgelist(df, source='citing_doi', target='cited_doi', create_using=nx.DiGraph())

In [84]:
succ_cnt = 0
pred_cnt = 0
for nd in g.nodes:
    if(len(list(g.successors(nd)))==0):
        succ_cnt += 1
    if(len(list(g.predecessors(nd)))==0):
        pred_cnt += 1
succ_cnt,pred_cnt

(2564, 5419)

In [97]:
adj = nx.linalg.graphmatrix.adjacency_matrix(g).toarray().T #we need to transpose


In [220]:
(adj.sum(axis=0)==0).sum(),(adj.sum(axis=1)==0).sum()

(2564, 5419)

In [135]:
k_out = adj.sum(axis=0).flatten()
k_in = adj.sum(axis=1).flatten()


In [271]:
factor = 1/k_out
factor[~np.isfinite(factor)]=0
T = adj*factor
dead_ends_mask = k_out == 0
d = 0.85
n = T.shape[0]
np.save('adjacency.npy', adj)
np.save('T_ver1.npy', T)
for i,flag in enumerate(dead_ends_mask):
    if(flag):
        T[:,i]=0
        T[i,i]=1
np.save('T_ver2.npy', T)
for i,flag in enumerate(dead_ends_mask):
    if(flag):
        T[:,i]=0
prank = np.ones(n)/n
iterations = 100
for t in range(iterations):
    new_pr = d*np.matmul(T,prank)+(1-d)/n
    prank = new_pr


<ipython-input-271-3ceb947657d5>:1: RuntimeWarning: divide by zero encountered in true_divide
  factor = 1/k_out


In [255]:
mask_deg = np.flip(np.argsort(k_in))

In [256]:
mask_rank = np.flip(np.argsort(prank))

In [257]:
mask_deg[:10]

array([ 4697,  8352, 15082,  2142,  2459,  2508, 15070,  4408,  4932,
        1023])

In [258]:
mask_rank[:10]

array([4697, 1699, 1698,  672, 2459, 1370, 1396, 2455, 8352, 8363])

In [259]:
nx_rank = np.array(list(dict(nx.pagerank(g)).values()))

In [260]:
mask_nx = np.flip(np.argsort(nx_rank))

In [261]:
mask_nx[:20]

array([4697, 2459, 1370, 2455, 1396, 8352,  672, 8363, 1023, 1624, 3553,
       5537, 3255,  452, 2142, 1167, 7423, 2471, 1699,  295])

In [262]:
mask_rank[:10]

array([4697, 1699, 1698,  672, 2459, 1370, 1396, 2455, 8352, 8363])

In [251]:
mask_deg[:20]

array([ 4697,  8352, 15082,  2142,  2459,  2508, 15070,  4408,  4932,
        1023, 10431, 15051,  2957, 13671,  8363,  1397, 15032,  2765,
        1370,  2764])

In [266]:
np.intersect1d(mask_rank[:20],mask_nx[:20] )

array([ 295,  452,  672, 1023, 1167, 1370, 1396, 1624, 1699, 2455, 2459,
       3255, 3553, 4697, 5537, 8352, 8363])